<h1>Обучение нейронной сети прямого распространения для прогнозирования ледового покрытия</h1>

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
data = pd.read_csv('../data/transformed_data.csv')
data.head()

,time,power_pump,temp_ice,temp_outside,hum_outside,temp_inside,hum_inside,max_motion,illuminance,CO2,temp_condensation,consumption_compressors,power_compressors,power_condensators,temp_supply_glycol,temp_return_glycol
0,2023-02-17 00:01:00,6844.409912,-3.578125,-7.95,79.0,7.15,38.225000,18.00,73.199997,425.500,13.103667,0.514583,30818.615547,6169.069941,-7.85,-5.31
1,2023-02-17 00:02:00,6860.609863,-3.609375,-7.95,79.0,7.15,38.750000,17.25,72.559998,426.875,12.223584,0.500000,30323.374219,6172.647412,-7.75,-5.31
2,2023-02-17 00:03:00,6856.255127,-3.619141,-7.95,79.0,7.15,39.174999,20.00,73.599997,427.000,7.812167,0.500000,29686.730156,4026.280020,-7.85,-5.31
3,2023-02-17 00:04:00,6870.135010,-3.611328,-7.95,79.0,7.15,38.700000,19.25,73.519997,429.000,7.431917,0.528906,31717.734297,4022.334907,-7.85,-5.31
4,2023-02-17 00:05:00,6868.975098,-3.603516,-7.95,79.0,7.15,38.474999,20.25,73.599997,430.500,11.624917,0.542448,32606.476172,5103.015059,-7.85,-5.31


In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
power_pump,364173.0,7752.440057,4089.333675,9.965715,5329.354980,6433.294922,11562.789551,16214.395020
temp_ice,364173.0,-3.872665,0.609632,-5.619792,-4.253606,-3.929688,-3.578125,-1.007812
temp_outside,364173.0,11.231066,8.702751,-19.801016,5.408898,12.861500,17.865081,30.899416
hum_outside,364173.0,69.265405,19.974126,16.016129,54.000000,73.050847,86.525000,100.000000
temp_inside,364173.0,9.499680,1.173811,5.606522,8.682143,9.594445,10.294697,13.487500
hum_inside,364173.0,56.599894,7.536656,17.275000,52.724999,56.800000,61.340476,79.475002
max_motion,364173.0,23.195631,18.794627,11.750000,18.000000,19.250000,21.000000,200.000000
illuminance,364173.0,60.427140,79.617248,0.000000,10.100000,67.520002,80.160000,1000.000000
CO2,364173.0,565.191667,143.565915,400.000000,438.416667,538.000000,657.000000,1458.750000
temp_condensation,364173.0,24.976893,7.219128,1.843333,19.359167,23.817500,30.905834,42.710625


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364173 entries, 0 to 364172
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   time                     364173 non-null  object 
 1   power_pump               364173 non-null  float64
 2   temp_ice                 364173 non-null  float64
 3   temp_outside             364173 non-null  float64
 4   hum_outside              364173 non-null  float64
 5   temp_inside              364173 non-null  float64
 6   hum_inside               364173 non-null  float64
 7   max_motion               364173 non-null  float64
 8   illuminance              364173 non-null  float64
 9   CO2                      364173 non-null  float64
 10  temp_condensation        364173 non-null  float64
 11  consumption_compressors  364173 non-null  float64
 12  power_compressors        364173 non-null  float64
 13  power_condensators       364173 non-null  float64
 14  temp